In [18]:
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

path = './cat-dog'

데이터 증식(Data argumentaion)을 위해 케라스에서 제공하는 이미지 제너레이터를 사용합니다.  
이미지의 위치를 조금 옮긴다거나, 회전, 좌우반전등을 했을 때 컴퓨터가 받아들이는 이미지는 전혀 다른것이 됩니다.  
이러한 변형을 줌으로써 학습 데이터를 늘리고, 이러한 변조에 강하게 모델을 학습시킬 수 있습니다.  

rescale은 이미지의 nomalization을 위해 사용합니다. 각 이미지별로 255로 나눈 값으로 데이터가 변형됩니다.  
rotation_range는 이미지의 최대 회전각을 지정합니다. 최대 20도까지 회전합니다.  
width,height shift_range는 이미지의 이동을 말합니다. 좌우, 위아래로 이미지의 이동하는 백분율을 지정합니다. (0.1은 10%)  
brightness_range는 이미지 밝기에 대한 내용입니다.  
horizontal_flip은 이미지의 수평 반전을 시켜줍니다.  
이 옵션의 경우 데이터셋의 이해가 필요합니다.  
예를 들면 MNIST 데이터셋의 경우 손글씨 데이터이기 때문에 수평 반전이 일어나면 안됩니다.  
validation_split은 검증세트의 비율을 지정해줍니다.

In [17]:
imageGenerator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[.2, .2],
    horizontal_flip=True,
    validation_split=.1
)

In [20]:
trainGen = imageGenerator.flow_from_directory(
    os.path.join(path, 'training_set'),
    target_size=(64, 64),
    subset='training'
)

validationGen = imageGenerator.flow_from_directory(
    os.path.join(path, 'training_set'),
    target_size=(64, 64),
    subset='validation'
)

Found 7205 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [22]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.summary()
# COMPILE
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
history = model.fit_generator(
    trainGen, 
    epochs=5,
    steps_per_epoch=trainGen.samples / 5, 
    validation_data=validationGen,
    validation_steps=trainGen.samples / 5,
)

Epoch 1/5
 789/1441 [===============>..............] - ETA: 6:28 - loss: 0.6877 - accuracy: 0.5375

KeyboardInterrupt: 

In [ ]:
testGenerator = ImageDataGenerator(
    rescale=1./255
)

testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(64, 64),
)

model.evaluate_generator(testGen)